In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)

from sklearn.model_selection import train_test_split

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

import numpy as np
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier, MLPRegressor
from collections import Counter


In [2]:
property_df = pd.read_csv("../data/curated/full_data.csv")

In [3]:
property_df.head(1)

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.0,5,3,3,House,mud brick magic,211021262,145.209992,-37.747366,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262.0


In [4]:
X = property_df
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
single_counts = X2[X2['count']==1].reset_index()['SA2_NAME21'].tolist() # can improve this later with bootstrapping
X = X[~X["SA2_NAME21"].isin(single_counts)]
X.head(1)

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.0,5,3,3,House,mud brick magic,211021262,145.209992,-37.747366,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262.0


In [5]:
y = X['cost_text'].tolist()
type(y[0])
# note: dropping desc head for now since it is not valuable yet
# SA2 CODE covered by SA2 Name
X.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", "cost_text"], axis=1, 
       inplace=True)
X.astype({ 
          "Median_tot_prsnl_inc_weekly":"float", 
          "Median_rent_weekly":"float",
          "Median_tot_fam_inc_weekly":"float", 
          "Median_tot_hhd_inc_weekly":"float", 
          "total_population": "int64",
          "duration_mins": "category", 
          "Bed":"int32", 
          "Bath":"int32", 
          "Park":"int32",
          "property_type":"category",
          "SA2_NAME21": "category",
          "Offence Count": "int32"}).dtypes

l = [(elem, type(elem)) for elem in X.loc[0]]
print(l)
X.head(1)

[(5, <class 'numpy.int64'>), (3, <class 'numpy.int64'>), (3, <class 'numpy.int64'>), ('House', <class 'str'>), (930.0, <class 'numpy.float64'>), (481.0, <class 'numpy.float64'>), (3051.0, <class 'numpy.float64'>), (2779.0, <class 'numpy.float64'>), (8781.0, <class 'numpy.float64'>), ('Warrandyte - Wonga Park', <class 'str'>), ('>50.0', <class 'str'>), (859.0, <class 'numpy.float64'>)]


/tmp/ipykernel_37206/3736008520.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", "cost_text"], axis=1,


,Bed,Bath,Park,property_type,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count
0,5,3,3,House,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0


In [6]:
type(y[0])

float

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
scaler = StandardScaler()
from sklearn.compose import ColumnTransformer

numeric_features = [ 
                  "Median_tot_prsnl_inc_weekly", 
                  "Median_rent_weekly",
                  "Median_tot_fam_inc_weekly", 
                  "Median_tot_hhd_inc_weekly", 
                  "total_population",
                  "Bed", 
                  "Bath", 
                  "Park",
                  "Offence Count"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_features)])

clf = MLPRegressor(max_iter=5000)
#this way we don't cheat. Read more on pipelines https://scikit-learn.org/stable/modules/compose.html
pipeline = Pipeline([('transformer', preprocessor), ('estimator', clf)]) # ('reduce_dim', PCA()),



In [ ]:
print('Cross-val standardised features R^2:', np.mean(cross_val_score(pipeline, X, y, cv=10)))
# 36%... not good. might need to perform PCA to improve, and further feature selection.
# can try grid search to tune hyperparams. 

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=X[['SA2_NAME21']])


clf.fit(preprocessor.fit_transform(X_train),y_train) # need to perform a train test split here
# need to do balanced split- i.e. one of the bootstrapping methods or something to make sure all suburbs have
# a good amount of representation. 


#print(clf.coefs_)
print('parameter shapes:',[p.shape for p in clf.coefs_])
print('num layers:', clf.n_layers_)
print('R^2 these params:', clf.score(preprocessor.transform(X_test), y_test)) # 34%


parameter shapes: [(9, 100), (100, 1)]
num layers: 3
R^2 these params: 0.33685884041474434


In [9]:
from sklearn.metrics import mean_squared_error
y_pred = clf.predict(preprocessor.transform(X_test))
print(mean_squared_error(y_test, y_pred))

61686.71971030945


## Some Notes
So we see that we only get an R^2 of 34%. This is before tuning hyperparams, but even then, it is difficult to believe changing hyperparameters will be a significant enough boost. Let us try another method. 

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

alphas = [np.power(10.0, i) for i in range(-7, 2)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, 
                                                          stratify=X[['SA2_NAME21']])


clf.fit(preprocessor.fit_transform(X_train), y_train)
print('MLP R^2 without tuning:', clf.score(preprocessor.transform(X_test), y_test))

hidden_sizes = [[220], [550, 220]]
#arguments of MLPClassifier and a list of values for them to search and find the best.
param_grid = {'alpha': alphas, 'hidden_layer_sizes':hidden_sizes}

# Make an appropriate scoring function

gs = GridSearchCV(estimator=clf,
                  param_grid=param_grid,
                  scoring='neg_mean_squared_error',
                  cv=3,
                  n_jobs=1, # Somehow verbose will not print if you do multithread
                  verbose=1 # More verbose = More detailed print
                  )

gs.fit(preprocessor.fit_transform(X_train), y_train)
best_params = gs.best_params_
print('best_params', best_params)

clf = MLPRegressor(max_iter=5000, **best_params)
clf.fit(preprocessor.fit_transform(X_train), y_train)
print('R^2 with best params:', clf.score(preprocessor.transform(X_test), y_test))

MLP acc without tuning: 0.44355131699049877
Fitting 3 folds for each of 18 candidates, totalling 54 fits
best_params {'alpha': 1e-06, 'hidden_layer_sizes': [550, 220]}
acc with best params: 0.41715583336392525


In [12]:


import pandas as pd
pd.DataFrame([gs.cv_results_['param_alpha'],
              gs.cv_results_['param_hidden_layer_sizes'],
              gs.cv_results_['mean_test_score'],
              gs.cv_results_['mean_fit_time'],
              gs.cv_results_['rank_test_score']], 
             index=['alpha','Hidden layer size','Mean test score', 'Mean fit time', 'Ranking'])



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
alpha,0.0,0.0,0.000001,0.000001,0.00001,0.00001,0.0001,0.0001,0.001,0.001,0.01,0.01,0.1,0.1,1.0,1.0,10.0,10.0
Hidden layer size,[220],"[550, 220]",[220],"[550, 220]",[220],"[550, 220]",[220],"[550, 220]",[220],"[550, 220]",[220],"[550, 220]",[220],"[550, 220]",[220],"[550, 220]",[220],"[550, 220]"
Mean test score,-48575.542502,-48640.61649,-48869.09414,-48330.961508,-48692.962324,-48968.969815,-48992.488505,-49031.911255,-48859.91633,-48701.787827,-48925.663187,-48894.414334,-48621.001326,-49584.692768,-48810.591275,-49212.972541,-48896.85161,-48965.943752
Mean fit time,71.638053,98.43627,50.176691,78.608315,67.138162,123.065611,59.910391,127.601986,64.570224,96.116447,68.860395,123.292212,74.478431,125.335948,65.272765,126.155659,58.309073,139.856324
Ranking,2,4,9,1,5,14,15,16,8,6,12,10,3,18,7,17,11,13


We see that the accuracy is slightly low, despite our hypter parameter tuning. It is likely that the sheer amount of data is making it difficult to build a good model. We could try dividing things into categories and building seperate models. 

In [9]:
import geopandas as gpd
# sf stands for shape file
australia_sf = gpd.read_file("../data/raw/shapefiles/Statistical_area_level2/SA2_2021_AUST_GDA2020.shp")
australia_sf.head(3)
# seperate suburbs that are in Victoria
vic_sf = australia_sf[australia_sf['STE_NAME21'] == 'Victoria']

# drop the null location ID values
vic_sf.dropna(inplace=True)
vic_sf.head(1)

/tmp/ipykernel_22895/2689031410.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vic_sf.dropna(inplace=True)


,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
644,201011001,Alfredton,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,52.7109,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5..."


In [11]:
vic_centroid_sf = vic_sf
vic_centroid_sf['geometry'] = vic_sf['geometry'].to_crs('+proj=cea').centroid.to_crs(vic_sf['geometry'].crs)
cbd_loc = vic_centroid_sf.where(vic_centroid_sf['SA2_NAME21'] == 'Melbourne CBD - North').dropna()['geometry']
cbd_loc

780    POINT (144.96089 -37.80918)
Name: geometry, dtype: geometry

In [18]:
# function taken from https://stackoverflow.com/a/56661833

from math import radians, cos, sin, asin, sqrt, inf

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# this function converts two floats to a point that is recognised by Shapely
def point_to_dist(row):
    dist = haversine(row["geometry"].y, row["geometry"].x, cbd_loc.y, cbd_loc.x)
    return dist


In [19]:
X = property_df
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
single_counts = X2[X2['count']==1].reset_index()['SA2_NAME21'].tolist() # can improve this later with bootstrapping
X = X[~X["SA2_NAME21"].isin(single_counts)]
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
X3 = vic_centroid_sf.join(X2, how='inner',on='SA2_NAME21')


X3['cbd_dist'] = X3.apply (lambda row: point_to_dist(row), axis=1)


In [20]:
X3.sort_values(by=['cbd_dist'], inplace=True)
X3

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry,count,cbd_dist
780,206041504,Melbourne CBD - North,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.5616,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96089 -37.80918),252,0.000000
781,206041505,Melbourne CBD - West,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,1.0051,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.95871 -37.81672),200,0.728392
779,206041503,Melbourne CBD - East,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.8020,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96860 -37.81277),138,0.917602
771,206041117,Carlton,0,No change,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,1.8187,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96781 -37.80049),218,1.103787
786,206041510,West Melbourne - Residential,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.8247,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.94977 -37.80857),67,1.237667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,212021293,Berwick - North,0,No change,21202,Casey - North,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,19.7480,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.34885 -38.02511),32,47.426478
978,211051283,Mount Evelyn,0,No change,21105,Yarra Ranges,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,16.9894,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.38822 -37.78726),1,47.559317
1010,212031558,Cranbourne East - South,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,5.3272,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.30423 -38.12247),13,47.691723
973,211051278,Lilydale - Coldstream,0,No change,21105,Yarra Ranges,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,109.3817,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.38884 -37.71375),9,48.377052


In [25]:
count = 0
BREAK = 2500
li = []
suburb_lists = []
for index, row in X3.iterrows():
    #print(row)
    count += int(row['count'])
    li.append(row["SA2_NAME21"])
    if len(suburb_lists) < 3: 
        if count >= BREAK:
            count = 0
            suburb_lists.append(li)
            li = []
suburb_lists.append(li)

In [34]:
sl = 50

numeric_features = [ 
                  "Median_tot_prsnl_inc_weekly", 
                  "Median_rent_weekly",
                  "Median_tot_fam_inc_weekly", 
                  "Median_tot_hhd_inc_weekly", 
                  "total_population",
                  "Bed", 
                  "Bath", 
                  "Park",
                  "Offence Count",
                  "cbd_dist"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_features)])

for i in range(len(suburb_lists)): 
    print(f"\n-- Beginning Ring {i} -- ")
    suburbs = suburb_lists[i]
    test_df = property_df.where(property_df['SA2_NAME21'].isin(suburbs)).dropna()

    y = test_df['cost_text'].tolist()
    print("\tnum training instances: ", len(y))
    # note: dropping desc head for now since it is not valuable yet
    # SA2 CODE covered by SA2 Name
    test_df.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", 
                  "cost_text", "SA2_NAME21"], axis=1, 
           inplace=True)
    test_df.astype({ 
              "Median_tot_prsnl_inc_weekly":"float", 
              "Median_rent_weekly":"float",
              "Median_tot_fam_inc_weekly":"float", 
              "Median_tot_hhd_inc_weekly":"float", 
              "total_population": "int64",
              "duration_mins": "category", 
              "Bed":"int32", 
              "Bath":"int32", 
              "Park":"int32",
              "property_type":"category",
              #"SA2_NAME21": "category",
              "Offence Count": "int32",
              "cbd_dist": "float"}).dtypes

    X_train, X_test, y_train, y_test = train_test_split(test_df, y, random_state=0, test_size=0.1)
                                                        #stratify=test_df[['SA2_NAME21']])
    clf = MLPRegressor(max_iter=10000,hidden_layer_sizes=[sl])

    clf.fit(preprocessor.fit_transform(X_train),y_train)  


    #print(clf.coefs_)
    print('\tparameter shapes:',[p.shape for p in clf.coefs_])
    print('\tnum layers:', clf.n_layers_)
    print('\tR^2 these params:', clf.score(preprocessor.transform(X_test), y_test)) 
    # 0.36, 0.44, 0.45, 0.51 R^2s- better. 


-- Beginning Ring 0 -- 
	num training instances:  2573
	parameter shapes: [(10, 50), (50, 1)]
	num layers: 3
	R^2 these params: 0.37171786689741204

-- Beginning Ring 1 -- 
	num training instances:  2498
	parameter shapes: [(10, 50), (50, 1)]
	num layers: 3
	R^2 these params: 0.43876423986317725

-- Beginning Ring 2 -- 
	num training instances:  2505
	parameter shapes: [(10, 50), (50, 1)]
	num layers: 3
	R^2 these params: 0.44421427710244665

-- Beginning Ring 3 -- 
	num training instances:  3268
	parameter shapes: [(10, 50), (50, 1)]
	num layers: 3
	R^2 these params: 0.5212904793647606


In [37]:
# removing the single counts increases accuracy by a significant amount (6%)
# todo: perform grid search to look for better params

sl = 100

numeric_features = [ 
                  "Median_tot_prsnl_inc_weekly", 
                  "Median_rent_weekly",
                  "Median_tot_fam_inc_weekly", 
                  "Median_tot_hhd_inc_weekly", 
                  "total_population",
                  "Bed", 
                  "Bath", 
                  "Park",
                  "Offence Count",
                  "cbd_dist"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_features)])

test_df = property_df

y = test_df['cost_text'].tolist()
print("\tnum training instances: ", len(y))
# note: dropping desc head for now since it is not valuable yet
# SA2 CODE covered by SA2 Name
test_df.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", 
              "cost_text", "SA2_NAME21"], axis=1, 
       inplace=True)
test_df.astype({ 
          "Median_tot_prsnl_inc_weekly":"float", 
          "Median_rent_weekly":"float",
          "Median_tot_fam_inc_weekly":"float", 
          "Median_tot_hhd_inc_weekly":"float", 
          "total_population": "int64",
          "duration_mins": "category", 
          "Bed":"int32", 
          "Bath":"int32", 
          "Park":"int32",
          "property_type":"category",
          #"SA2_NAME21": "category",
          "Offence Count": "int32",
          "cbd_dist": "float"}).dtypes

X_train, X_test, y_train, y_test = train_test_split(test_df, y, random_state=0, test_size=0.1)
                                                    #stratify=test_df[['SA2_NAME21']])
clf = MLPRegressor(max_iter=10000,hidden_layer_sizes=[sl])

clf.fit(preprocessor.fit_transform(X_train),y_train)  


#print(clf.coefs_)
print('\tparameter shapes:',[p.shape for p in clf.coefs_])
print('\tnum layers:', clf.n_layers_)
print('\tR^2 these params:', clf.score(preprocessor.transform(X_test), y_test)) 
# 0.36, 0.44, 0.45, 0.51 R^2s- better. 

KeyError: 'cost_text'